In [10]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
sys.path.append(root_path)

In [38]:
from decimal import Decimal

# Controller configuration
connector_name = "kucoin"
trading_pair = "WLD-USDT"
total_amount_quote = 1000
trade_cost = 0.0006
buy_spreads = [Decimal("0.002")]
sell_spreads = [Decimal("0.002")]
buy_amounts_pct = [Decimal("1")]
sell_amounts_pct = [Decimal("1")]
executor_refresh_time = 60
cooldown_time = 3600
top_executor_refresh_time = 60
executor_activation_bounds = [Decimal("0.01")]
dca_spreads = [Decimal("0."), Decimal("0.002"), Decimal("0.008"), Decimal("0.01"), Decimal("0.015"), Decimal("0.02")]
dca_amounts = [Decimal("0.1"), Decimal("0.2"), Decimal("0.3"), Decimal("0.4"), Decimal("0.5"), Decimal("0.6")]

# Triple barrier configuration
stop_loss = Decimal("0.015")
take_profit = Decimal("0.03")
time_limit = 60 * 60 * 12 # 12 hours
trailing_stop_activation_price_delta = Decimal("0.008")
trailing_stop_trailing_delta = Decimal("0.004")

In [39]:
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from quants_lab.controllers.market_making.dman_maker_v2 import DManMakerV2Config, DManMakerV2


# Creating the instance of the configuration and the controller
config = DManMakerV2Config(
    connector_name=connector_name,
    trading_pair=trading_pair,
    total_amount_quote=total_amount_quote,
    buy_spreads=buy_spreads,
    sell_spreads=sell_spreads,
    buy_amounts_pct=buy_amounts_pct,
    sell_amounts_pct=sell_amounts_pct,
    executor_refresh_time=executor_refresh_time,
    cooldown_time=cooldown_time,
    top_executor_refresh_time=top_executor_refresh_time,
    executor_activation_bounds=executor_activation_bounds,
    stop_loss=stop_loss,
    take_profit=take_profit,
    time_limit=time_limit,
    trailing_stop=TrailingStop(activation_price=trailing_stop_activation_price_delta,
                               trailing_delta=trailing_stop_trailing_delta),
    dca_spreads=dca_spreads,
    dca_amounts=dca_amounts
)

In [47]:
import pandas as pd
from hummingbot.strategy_v2.backtesting import MarketMakingBacktesting

backtesting_engine = MarketMakingBacktesting()

# Backtest period
start = "2024-01-01"
end = "2024-01-10"
backtesting_resolution = "1m"

start_time = pd.to_datetime(start).timestamp() * 1e3
end_time = pd.to_datetime(end).timestamp() * 1e3
backtesting_results = await backtesting_engine.run_backtesting(
            controller_config=config, trade_cost=trade_cost,
            start=int(start_time), end=int(end_time),
            backtesting_resolution=backtesting_resolution)

2024-05-21 18:53:41,626 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x3094482e0>
2024-05-21 18:53:41,627 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x30ce441c0>, 55330.4882885)]']
connector: <aiohttp.connector.TCPConnector object at 0x309448ca0>


In [48]:
# Let's see what is inside the backtesting results
backtesting_results.keys()

dict_keys(['executors', 'results', 'processed_data'])

In [49]:
# now let's analyze each of the dataframes

# 1. The processed data: this is the data that we are going to use to generate the signal
backtesting_results["processed_data"]

{'reference_price': Decimal('2.497599999999999820232687852694652974605560302734375'),
 'spread_multiplier': Decimal('1'),
 'features':        timestamp_bt  open_bt  high_bt  low_bt  close_bt  volume_bt  \
 0     1704067200000   3.6272   3.6335  3.6268    3.6335  1247.4443   
 1     1704067260000   3.6333    3.639  3.6317     3.639   373.5355   
 2     1704067320000   3.6476    3.657  3.6476     3.657   504.2724   
 3     1704067380000   3.6601   3.6622  3.6586    3.6622    43.5494   
 4     1704067440000   3.6622   3.6622  3.6622    3.6622          0   
 ...             ...      ...      ...     ...       ...        ...   
 12964 1704844560000   2.4965   2.4965   2.495     2.495     5.1705   
 12965 1704844620000   2.4978   2.5025  2.4978    2.5025    37.5497   
 12966 1704844680000   2.5025   2.5025  2.5025    2.5025          0   
 12967 1704844740000   2.5027   2.5034  2.5006    2.5006    63.5077   
 12968 1704844800000   2.5026   2.5026  2.4976    2.4976  1009.2771   
 
        quot

In [50]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
backtesting_results["executors"]

[ExecutorInfo(id='ATh9By7hcv5V8pLarj4Cy69kKuYgBpNYag1QS428cgjc', timestamp=1704067200000.0, type='dca_executor', close_timestamp=1704067320000.0, close_type=<CloseType.EARLY_STOP: 5>, status=<RunnableStatus.TERMINATED: 4>, config=DCAExecutorConfig(id='ATh9By7hcv5V8pLarj4Cy69kKuYgBpNYag1QS428cgjc', type='dca_executor', timestamp=1704067200000.0, controller_id='main', connector_name='kucoin', trading_pair='WLD-USDT', side=<TradeType.BUY: 1>, leverage=20, amounts_quote=[Decimal('23.80952380952380952380952381'), Decimal('47.52380952380952380952380955'), Decimal('70.85714285714285714285714287'), Decimal('94.28571428571428571428571430'), Decimal('117.2619047619047619047619048'), Decimal('140.0000000000000000000000000')], prices=[Decimal('3.626233000000000173583529495'), Decimal('3.618980534000000173236362436'), Decimal('3.597223136000000172194861259'), Decimal('3.589970670000000171847694200'), Decimal('3.571839505000000170979776553'), Decimal('3.553708340000000170111858905')], take_profit=De

In [51]:
# 3. The results dataframe: this is the dataframe that contains the information of the pnl of the strategy
backtesting_results["results"]

{'net_pnl': 0.042269981264627964,
 'net_pnl_quote': 42.269981264627965,
 'total_executors': 414,
 'total_executors_with_position': 99,
 'total_volume': 70749.14285714286,
 'total_long': 52,
 'total_short': 47,
 'close_types': {'EARLY_STOP': 314,
  'STOP_LOSS': 47,
  'TAKE_PROFIT': 48,
  'TIME_LIMIT': 5},
 'accuracy_long': 0.46153846153846156,
 'accuracy_short': 0.5531914893617021,
 'total_positions': 99,
 'accuracy': 0.5050505050505051,
 'max_drawdown_usd': -52.26680777736686,
 'max_drawdown_pct': -0.052671130099730776,
 'sharpe_ratio': 0.479129045415411,
 'profit_factor': 1.1213189350926154,
 'win_signals': 50,
 'loss_signals': 49}

In [52]:
from frontend.visualization.backtesting import create_backtesting_figure

fig = create_backtesting_figure(
        df=backtesting_results["processed_data"]["features"],
        executors=backtesting_results["executors"],
        config=config.dict())

In [53]:
fig

### Scatter of PNL per Trade
This bar chart illustrates the PNL for each individual trade. Positive PNLs are shown in green and negative PNLs in red, providing a clear view of profitable vs. unprofitable trades.


In [34]:
import plotly.express as px

executors_df = pd.DataFrame(ei.to_dict() for ei in backtesting_results["executors"])
executors_df["profitable"] = executors_df["net_pnl_quote"] > 0

fig = px.scatter(executors_df,  x="timestamp", y='net_pnl_quote', title='PNL per Trade',
             color='profitable', color_continuous_scale=['red', 'green'])
fig.show()

### Scatter Plot of Volume vs. PNL
This scatter plot explores the relationship between the trade volume and the PNL for each trade. It can reveal if larger volumes are associated with higher profits or losses.


In [36]:
fig = px.scatter(executors_df, x='filled_amount_quote', y='net_pnl_quote', title='Trade Volume vs. PNL')
fig.show()

### Histogram of PNL Distribution
The histogram displays the distribution of PNL values across all trades. It helps in understanding the frequency and range of profit and loss outcomes.


In [37]:
fig = px.histogram(executors_df, x='net_pnl_quote', title='PNL Distribution')
fig.show()
